In [8]:
import json
import csv
import os
import pandas as pd
import numpy as np
import random
import copy
from datetime import datetime

json_path = './yelp/'
data_path = './dataset/'

## Review 데이터 가공

user_id 및 business_id 추출, 정렬

In [9]:
# 파일 경로 설정
json_file_path = json_path + 'review.json'

# 유저 ID와 비즈니스 ID를 매핑하기 위한 딕셔너리
user_id_mapping = {}
business_id_mapping = {}
next_user_index = 0
next_business_index = 0

# 최종 데이터를 리스트에 저장
sort_data = []

# JSON 파일을 읽고 처리
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    for line in json_file:
        data = json.loads(line)
        user_id = data['user_id']
        business_id = data['business_id']

        # 유저 ID 매핑 업데이트
        if user_id not in user_id_mapping:
            user_id_mapping[user_id] = next_user_index
            next_user_index += 1
        # 비즈니스 ID 매핑 업데이트
        if business_id not in business_id_mapping:
            business_id_mapping[business_id] = next_business_index
            next_business_index += 1

        # 매핑된 ID로 데이터 저장
        sort_data.append([user_id_mapping[user_id], business_id_mapping[business_id]])

# 데이터를 정렬하고 새 파일에 저장
sort_data.sort(key=lambda x: (x[0], x[1]))  # 먼저 user_id, 다음 business_id 기준으로 정렬\

print("모든 데이터가 user_id와 business_id를 기준으로 정렬되어 저장되었습니다.")


모든 데이터가 user_id와 business_id를 기준으로 정렬되어 저장되었습니다.


In [10]:
# 필라델피아에 있는 비즈니스 ID를 추출하기 위한 파일 경로
business_json_path = json_path + 'business.json'

# 필라델피아 비즈니스 ID 저장을 위한 세트
philadelphia_business_ids = set()

# business.json 파일 읽기
with open(business_json_path, 'r', encoding='utf-8') as file:
    for line in file:
        business_data = json.loads(line)
        if business_data['city'] == 'Philadelphia':
            philadelphia_business_ids.add(business_data['business_id'])

# 필라델피아 비즈니스 리뷰만 포함하는 데이터 추출
philadelphia_sort_data = [
    [user_id, business_id] for user_id, business_id in sort_data
    if business_id_mapping.get(business_id) in philadelphia_business_ids
]

# 필요하다면 philadelphia_sort_data를 파일에 저장할 수 있습니다.

print("필라델피아의 비즈니스 리뷰 데이터가 준비되었습니다.")

필라델피아의 비즈니스 리뷰 데이터가 준비되었습니다.


리뷰가 10개 이상인 유저만 필터링

In [11]:
# sort_data DataFrame으로 변환
sort_data = pd.DataFrame(sort_data, columns=['user_id', 'business_id'])

# 각 ID의 출현 빈도를 계산
frequency = sort_data['user_id'].value_counts()

# 출현 빈도가 10 이상인 ID만 필터링
valid_ids = frequency[frequency >= 10].index

# 필터링된 ID에 해당하는 데이터만 선택
filtered_data = sort_data[sort_data['user_id'].isin(valid_ids)]

# 혹은 출력하여 확인
print(filtered_data)

         user_id  business_id
0              0            0
1              0          473
2              0          927
3              0        15261
4              0        15753
...          ...          ...
6648468  1676360       114399
6648469  1676360       114399
6648470  1676360       114399
6648471  1676360       114399
6648472  1676360       114399

[3303811 rows x 2 columns]


user_id, business_id 인덱스 매핑

In [12]:
philadelphia = []

# business.json 파일 읽기
with open(business_json_path, 'r', encoding='utf-8') as file:
    for line in file:
        business_data = json.loads(line)
        if business_data['city'] == 'Philadelphia':
            philadelphia_business_id = business_data['business_id']
            philadelphia_latitude = business_data['latitude']
            philadelphia_longitude = business_data['longitude']
            philadelphia.append({'business_id': philadelphia_business_id, 
                                 'latitude': philadelphia_latitude, 
                                 'longitude': philadelphia_longitude})
            
philadelphia = pd.DataFrame(philadelphia)

philadelphia['business_id'] = philadelphia['business_id'].map(business_id_mapping)


# 유저 ID에 대해 고유한 ID를 정렬된 리스트로 추출하고 0부터 시작하는 새로운 인덱스를 매핑
unique_user_ids = sorted(filtered_data['user_id'].unique())
user_id_mapping = {id: index for index, id in enumerate(unique_user_ids)}
filtered_data['user_id'] = filtered_data['user_id'].map(user_id_mapping)

# 비즈니스 ID에 대해 고유한 ID를 정렬된 리스트로 추출하고 0부터 시작하는 새로운 인덱스를 매핑
print(filtered_data)
unique_business_ids = sorted(filtered_data['business_id'].unique())
business_id_mapping_uni = {id: index for index, id in enumerate(unique_business_ids)}
filtered_data['business_id'] = filtered_data['business_id'].map(business_id_mapping_uni)

philadelphia['business_id'] = philadelphia['business_id'].map(business_id_mapping)

# CSV로 저장
philadelphia.to_csv('poi.csv', index=False)

# 중복 제거
indexing_data = filtered_data.drop_duplicates(subset=['user_id', 'business_id'])

# # 최종 데이터 프레임 출력하여 확인
# print(indexing_data)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_17500\2167955209.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['user_id'] = filtered_data['user_id'].map(user_id_mapping)


         user_id  business_id
0              0            0
1              0          473
2              0          927
3              0        15261
4              0        15753
...          ...          ...
6648468   117369       114399
6648469   117369       114399
6648470   117369       114399
6648471   117369       114399
6648472   117369       114399

[3303811 rows x 2 columns]


C:\Users\Administrator\AppData\Local\Temp\ipykernel_17500\2167955209.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['business_id'] = filtered_data['business_id'].map(business_id_mapping_uni)


In [ ]:
philadelphia['business_id'] = philadelphia['business_id'].map(business_id_mapping)

# CSV로 저장
philadelphia.to_csv('poi.csv', index=False)

train 및 test 데이터 분리

In [13]:
# drop_duplicates 함수를 사용하여, 각 사용자 ID와 아이템 ID 조합의 첫 번째 등장만 유지합니다.
first_data = indexing_data.drop_duplicates(subset=['user_id', 'business_id'])

# 각 사용자별로 첫 번째 데이터를 테스트 데이터로, 나머지를 트레이닝 데이터로 분리
train_data = []
test_data = []

# 사용자별로 그룹화하여 처리
for user_id, group in indexing_data.groupby('user_id'):
    test_data.append(group.iloc[0].tolist())  # 첫 번째 행을 테스트 데이터에 추가, 리스트로 변환
    train_data.extend(group.iloc[1:].values.tolist())  # 나머지 행을 트레이닝 데이터에 추가, 리스트로 변환

# 리스트를 DataFrame으로 변환
train_df = pd.DataFrame(train_data, columns=['user_id', 'business_id'])
test_df = pd.DataFrame(test_data, columns=['user_id', 'business_id'])

# CSV 파일로 저장
train_df.to_csv(data_path + 'train.csv', index=False, header=False, sep = '\t')
test_df.to_csv(data_path + 'test.csv', index=False, header=False, sep = '\t')

print("'train.csv'와 'test.csv' 파일이 생성되었습니다.")

'train.csv'와 'test.csv' 파일이 생성되었습니다.


negative 데이터 생성

In [14]:
# 유저와 아이템 집합 구하기
users = indexing_data['user_id'].unique()
businesses = indexing_data['business_id'].unique()

# 결과를 저장할 딕셔너리
neg_dict = {}

# 각 유저마다 가지고 있지 않은 아이템 찾기
for user in users:
    user_businesses = indexing_data[indexing_data['user_id'] == user]['business_id'].unique()
    not_having_businesses = np.setdiff1d(businesses, user_businesses)
    if len(not_having_businesses) > 100:
        sampled_businesses = np.random.choice(not_having_businesses, 100, replace=False)
    else:
        sampled_businesses = not_having_businesses
    neg_dict[user] = ','.join([str(item) for item in sampled_businesses])

# 결과를 CSV 파일로 저장
with open(data_path + 'test_neg.csv', 'w') as f:
    for user, businesses in neg_dict.items():
        f.write(f"{businesses}\n")

negative 데이터 생성

In [15]:
# 필요한 파일을 읽고 쓰기 위해 open 함수를 사용
with open(data_path + 'test_neg.csv', 'r') as neg_file, open(data_path + 'test.csv', 'r') as test_file, open(data_path + 'test_neg_100.csv', 'w') as result_file:
    neg_lines = neg_file.readlines()
    test_lines = test_file.readlines()
    
    for neg_line, test_line in zip(neg_lines, test_lines):
        user_id, neg_data = neg_line.strip().split(',', 1)
        test_index, test_value = test_line.strip().split('\t')
        test_tuple = f'({test_index},{test_value})'
        
        neg_data_formatted = '\t'.join(neg_data.split(','))
        
        result_line = test_tuple + '\t' + neg_data_formatted + '\n'
        result_file.write(result_line)

print("파일 'test_neg_100.csv'가 생성되었습니다.")

파일 'test_neg_100.csv'가 생성되었습니다.


In [16]:
# business_json_path = json_path + 'business.json'

# # JSON 파일 불러오기
# with open(business_json_path, 'r', encoding='utf-8') as f:
#     data = json.load(f)

# # 'city'가 'Philadelphia'인 데이터 필터링
# filtered_data = [entry for entry in data if entry.get('city') == 'Philadelphia']

# # 필요한 열만 선택
# selected_data = [{'business_id': entry['business_id'], 
#                   'latitude': entry['latitude'], 
#                   'longitude': entry['longitude']} for entry in filtered_data]

# # business_id를 매핑하여 index로 변경
# unique_business_ids = [entry['business_id'] for entry in selected_data]
# business_id_mapping = {id: index for index, id in enumerate(unique_business_ids)}

# # business_id를 index로 변경
# for entry in selected_data:
#     entry['business_id'] = business_id_mapping[entry['business_id']]

# # DataFrame 생성
# df = pd.DataFrame(selected_data)

# # CSV로 저장
# df.to_csv('poi.csv', index=False)


JSONDecodeError: Extra data: line 2 column 1 (char 434)